In [10]:
import numpy as np
import scipy.sparse as sps
import h5py
from termcolor import colored

def decode_str(hdf5_str):
    return hdf5_str.tobytes().decode('utf-16')

filename = 'data/Head-and-Neck_02.mat'

f = h5py.File(filename)
print(colored('Patient: ' + decode_str(f['patient']['Identifier'][:]), 'blue'))

Patient: Head-and-Neck 02


In [11]:
class Region:
    def __init__(self, name):
        self.name = name
        self.D = None # Dose deposition matrix (called A in TROTS)

    def compute_dose(self, x):
        self.dose = self.D.dot(x)
        self.min = self.dose.min()
        self.mean = self.dose.mean()
        self.max = self.dose.max()

def load_rois():
    print('Loading dose deposition matrices...')
    
    rois = {}
    for ref in f['patient']['StructureNames'][:]:
        name = decode_str(f[ref[0]][:])
        rois[name] = Region(name)

    # Get the dose matrix for each ROI
    n_mats = f['data']['matrix']['A'].shape[0]
    for i in range(n_mats):
        name = decode_str(f[f['data']['matrix']['Name'][i,0]][:])

        if name in rois:
            roi = rois[name]
            A = f[f['data']['matrix']['A'][i,0]]
            
            if isinstance(A, h5py.Group): # It's a sparse matrix
                data = np.array(A['data']).ravel()
                ir = np.array(A['ir']).ravel()
                jc = np.array(A['jc']).ravel()
                n_voxels = A.attrs.get('MATLAB_sparse')
                n_beamlets = jc.size - 1
                shape = (n_voxels, n_beamlets)
                
                roi.D = sps.csc_matrix((data, ir, jc), shape=shape)
                
                print(colored(f'  {name} -> {shape} (sparse)', 'green'))
            elif isinstance(A, h5py.Dataset): # It's a dense matrix
                roi.D = A[:].T
                shape = roi.D.shape
                print(colored(f'  {name} -> {shape} (dense)', 'green'))
            else:
                print(colored(f'  {name} -> Not processed!', 'magenta'))
        else:
            print(colored(f'  {name} -> Ignored', 'yellow'))

    # Check that all regions have dose matrices
    for name, roi in rois.items():
        if roi.D is None:
            print(colored(f'  {name} -> No dose matrix!', 'red'))
    
    return rois

rois = load_rois()

Loading dose deposition matrices...
  PTV 0-46 Gy -> (5167, 6331) (dense)
  Spinal Cord -> (3181, 6331) (sparse)
  Brainstem -> (3126, 6331) (sparse)
  PTV Shell 15 mm -> (4724, 6331) (sparse)
  PTV Shell 30 mm -> (4729, 6331) (sparse)
  PTV Shell 40 mm -> (4787, 6331) (sparse)
  PTV Shell 5 mm -> (4839, 6331) (sparse)
  Patient -> (10600, 6331) (sparse)
  PTV Shell 0 mm -> (4908, 6331) (sparse)
  Smoothing Linear -> Ignored
  Smoothing Quadratic -> Ignored
  Parotid (R) -> (3315, 6331) (sparse)
  Parotid (R) (mean) -> Ignored
  Parotid (L) -> (3165, 6331) (sparse)
  Parotid (L) (mean) -> Ignored
  SMG (R) -> (1487, 6331) (dense)
  SMG (R) (mean) -> Ignored
  SMG (L) -> (1675, 6331) (sparse)
  SMG (L) (mean) -> Ignored
  Oral Cavity -> (5325, 6331) (sparse)
  Oral Cavity (mean) -> Ignored
  External Ring 20 mm -> (5409, 6331) (sparse)
  Larynx -> (5075, 6331) (dense)
  Larynx (mean) -> Ignored
  MCS -> (1621, 6331) (dense)
  MCS (mean) -> Ignored
  MCM -> (990, 6331) (dense)
  MCM (mea

In [16]:
def compute_dose(x=x):
    for name, roi in rois.items():
        roi.compute_dose(x)

def dose_table():
    print(colored('Patient: ' + decode_str(f['patient']['Identifier'][:]), 'blue'))
    print()
    print(colored('{:20s} {:>9s} {:>9s} {:>9s}'.format('Region of Interest', 'Min.', 'Mean', 'Max.'), attrs=["bold"]))
    print(colored('-'*50, 'blue'))

    for name, roi in rois.items():
        print('{:20s} {:9.2f} {:9.2f} {:9.2f}'.format(name, roi.min, roi.mean, roi.max))

x = f['solutionX'][:].ravel()
compute_dose()
dose_table()

Patient: Head-and-Neck 02

Region of Interest        Min.      Mean      Max.
--------------------------------------------------
Patient                   0.00      0.00      0.00
Spinal Cord               0.00      0.00      0.00
Parotid (R)               0.00      0.00      0.00
Parotid (L)               0.00      0.00      0.00
SMG (R)                   0.00      0.00      0.00
SMG (L)                   0.00      0.00      0.00
MCS                       0.00      0.00      0.00
MCM                       0.00      0.00      0.00
MCI                       0.00      0.00      0.00
MCP                       0.00      0.00      0.00
Oesophagus                0.00      0.00      0.00
Brainstem                 0.00      0.00      0.00
Oral Cavity               0.00      0.00      0.00
Larynx                    0.00      0.00      0.00
PTV 0-46 Gy               0.00      0.00      0.00
PTV Shell 15 mm           0.00      0.00      0.00
PTV Shell 30 mm           0.00      0.00      0.00
PTV 

In [17]:
from enum import Enum
from dataclasses import dataclass

class CostFunction(Enum):
    Linear = 1
    Quadratic = 2
    gEUD = 3
    LTCP = 4
    DVH = 5
    Chain = 6

class Function:
    def __init__(self, name, type, is_constraint, is_minimize, target, is_scalar, priority):
        self.name = name
        self.type = CostFunction(type)
        self.minimum = None
        self.mean = None
        self.maximum = None
        self.target = target
        self.priority = priority

        if is_constraint:
            self.kind = 'Constraint'
        else:
            self.kind = 'Objective'

        if self.type.name == 'Linear':
            if self.kind == 'Objective':
                if is_scalar and is_minimize:
                    self.direction = 'mean'
                else:
                    self.direction = 'maximum'
            else:
                self.direction = 'maximum' if is_minimize else 'minimum'
        else:
            self.direction = ''
                
    def __repr__(self):
        return '{:20s} {} {:16s} {:>3d} {:8.2f}'.format(self.name, 
            colored('Const.', 'yellow') if self.kind == 'Constraint' else colored('Objec.', 'cyan'), 
            (self.type.name + ' (' + self.direction + ')') if self.direction != '' else self.type.name,
            self.priority, self.target)

def load_problem():
    print(colored('Patient: ' + decode_str(f['patient']['Identifier'][:]), 'blue'))
    print()
    print(colored('Region of Interest   Kind   Function         Pr.   Target  Current', attrs=["bold"]))
    print(colored('------------------------------------------------------------------', 'blue'))
    
    for i in range(f['problem']['dataID'].shape[0]):
        name = decode_str(f[f['problem']['Name'][i][:][0]][:])
        type = f[f['problem']['Type'][i][0]][0][0]
        is_constraint = f[f['problem']['IsConstraint'][i][0]][0][0] == 1
        is_minimize = f[f['problem']['Minimise'][i][0]][0][0] == 1
        target = f[f['problem']['Objective'][i][0]][0][0]
        dataID = int(f[f['problem']['dataID'][i][0]][0][0]) - 1
        is_scalar = isinstance(f[f['data']['matrix']['A'][dataID,0]], h5py.Dataset)
        #is_sufficient = isinstance(f[f['problem']['Sufficient'][i][0]][0], np.ndarray)
        priority = int(f[f['problem']['Priority'][i][0]][0][0])
              
        function = Function(name, type, is_constraint, is_minimize, target, is_scalar, priority)
        print(function, end='')

        if name in rois:
            roi = rois[name]
            if function.type == CostFunction.Linear:
                if function.direction == 'minimum':
                    value = roi.min
                    if value > function.target:
                        color = 'green'
                    else:
                        color = 'red'
                elif function.direction == 'mean':
                    value = roi.mean
                    if value < function.target:
                        color = 'green'
                    else:
                        color = 'red'
                else:
                    value = roi.max
                    if value < function.target:
                        color = 'green'
                    else:
                        color = 'red'
                print(colored(' {:8.2f}'.format(value), color))
            else:
                print()
        else:
            print()
        
load_problem()

Patient: Head-and-Neck 02

Region of Interest   Kind   Function         Pr.   Target  Current
------------------------------------------------------------------
PTV 0-46 Gy          Const. Linear (maximum)   0    48.30     0.00
Spinal Cord          Const. Linear (maximum)   0    38.00     0.00
Brainstem            Const. Linear (maximum)   0    38.00     0.00
Patient              Const. Linear (maximum)   0    48.30     0.00
PTV Shell 0 mm       Const. Linear (maximum)   0    46.00     0.00
Parotid (R)          Const. Linear (maximum)   0    48.30     0.00
Parotid (L)          Const. Linear (maximum)   0    48.30     0.00
SMG (R)              Const. Linear (maximum)   0    48.30     0.00
SMG (L)              Const. Linear (maximum)   0    48.30     0.00
Oral Cavity          Const. Linear (maximum)   0    48.30     0.00
Larynx               Const. Linear (maximum)   0    48.30     0.00
MCS                  Const. Linear (maximum)   0    48.30     0.00
MCM                  Const. Linear 

In [43]:
print(type(x), x.shape[0], x.dtype, x.min(), x.mean(), x.max(), x.std())

<class 'numpy.ndarray'> 6331 float64 1.605611122457795e-08 222.62117559330832 2026.4191580421273 219.9125848174818


In [44]:
new_x = np.random.uniform(low=x.min(), high=x.max(), size=x.shape[0])
compute_dose(new_x)
load_problem()

Patient: Head-and-Neck 02

Region of Interest   Kind   Function         Pr.   Target  Current
------------------------------------------------------------------
PTV 0-46 Gy          Const. Linear (maximum)   0    48.30   235.20
Spinal Cord          Const. Linear (maximum)   0    38.00   225.66
Brainstem            Const. Linear (maximum)   0    38.00    60.76
Patient              Const. Linear (maximum)   0    48.30   230.06
PTV Shell 0 mm       Const. Linear (maximum)   0    46.00   231.92
Parotid (R)          Const. Linear (maximum)   0    48.30   197.47
Parotid (L)          Const. Linear (maximum)   0    48.30   204.45
SMG (R)              Const. Linear (maximum)   0    48.30   225.53
SMG (L)              Const. Linear (maximum)   0    48.30   218.75
Oral Cavity          Const. Linear (maximum)   0    48.30   193.16
Larynx               Const. Linear (maximum)   0    48.30   233.38
MCS                  Const. Linear (maximum)   0    48.30   224.62
MCM                  Const. Linear 

In [45]:
new_x = np.random.uniform(low=x.min(), high=x.max(), size=x.shape[0])
compute_dose(new_x)
load_problem()

Patient: Head-and-Neck 02

Region of Interest   Kind   Function         Pr.   Target  Current
------------------------------------------------------------------
PTV 0-46 Gy          Const. Linear (maximum)   0    48.30   230.61
Spinal Cord          Const. Linear (maximum)   0    38.00   210.78
Brainstem            Const. Linear (maximum)   0    38.00    59.12
Patient              Const. Linear (maximum)   0    48.30   231.72
PTV Shell 0 mm       Const. Linear (maximum)   0    46.00   231.05
Parotid (R)          Const. Linear (maximum)   0    48.30   204.31
Parotid (L)          Const. Linear (maximum)   0    48.30   196.88
SMG (R)              Const. Linear (maximum)   0    48.30   222.14
SMG (L)              Const. Linear (maximum)   0    48.30   207.44
Oral Cavity          Const. Linear (maximum)   0    48.30   185.86
Larynx               Const. Linear (maximum)   0    48.30   226.63
MCS                  Const. Linear (maximum)   0    48.30   210.61
MCM                  Const. Linear 

In [42]:
new_x = np.ones(x.shape[0])
compute_dose(new_x)
load_problem()

Patient: Head-and-Neck 02

Region of Interest   Kind   Function         Pr.   Target  Current
------------------------------------------------------------------
PTV 0-46 Gy          Const. Linear (maximum)   0    48.30     0.21
Spinal Cord          Const. Linear (maximum)   0    38.00     0.21
Brainstem            Const. Linear (maximum)   0    38.00     0.06
Patient              Const. Linear (maximum)   0    48.30     0.21
PTV Shell 0 mm       Const. Linear (maximum)   0    46.00     0.21
Parotid (R)          Const. Linear (maximum)   0    48.30     0.20
Parotid (L)          Const. Linear (maximum)   0    48.30     0.20
SMG (R)              Const. Linear (maximum)   0    48.30     0.21
SMG (L)              Const. Linear (maximum)   0    48.30     0.21
Oral Cavity          Const. Linear (maximum)   0    48.30     0.18
Larynx               Const. Linear (maximum)   0    48.30     0.21
MCS                  Const. Linear (maximum)   0    48.30     0.20
MCM                  Const. Linear 